### Loading packages and data

In [26]:
import sqlite3
import pandas as pd
import json
from itertools import chain
from random import sample
import numpy as np
import os
from bs4 import BeautifulSoup
import string
import re
import xmltodict
from xml.etree import ElementTree as ET
import requests
import json

In [4]:
conn = sqlite3.connect("/Volumes/data_commons-Ubel/gfm/gfm.db")

In [3]:
path_to_json = '/Volumes/data_commons-Ubel/gfm/scraped_data'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [22]:
for json_file in json_files:
    json_file_path = os.path.join(path_to_json, json_file)
    with open (json_file_path, "r") as f:
        data = json.load(f)
    #send to db
    make_scrape_tb(data)
    feed = make_feed_tb(data)
    make_donation_tb(data)
    make_team_member_tb(data, feed)
    make_comment_tb(data)
    make_update_tb(data)
    print(json_file)

data_301.json
data_197.json
data_11.json
data_474.json
data_406.json
data_444.json
data_164.json
data_322.json
data_319.json
data_100.json
data_37.json
data_85.json
data_4.json
data_10.json
data_278.json
data_154.json
data_103.json
data_321.json
data_382.json
data_91.json
data_22.json
data_135.json
data_109.json
data_27.json
data_451.json
data_369.json
data_137.json
data_172.json
data_361.json
data_386.json
data_15.json
data_478.json
data_170.json
data_279.json
data_123.json
data_20.json
data_250.json
data_455.json
data_359.json
data_449.json
data_304.json
data_17.json
data_18.json
data_237.json
data_52.json
data_223.json
data_146.json
data_446.json
data_330.json
data_176.json
data_74.json
data_47.json
data_125.json
data_214.json
data_87.json
data_495.json
data_89.json
data_262.json
data_347.json
data_477.json
data_241.json
data_54.json
data_393.json
data_227.json
data_469.json
data_236.json
data_281.json
data_112.json
data_481.json
data_209.json
data_65.json
data_129.json
data_277.jso

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': unable to open database file

### Establish database connection

In [121]:
conn = sqlite3.connect("/Users/sd363/Desktop/crowdfunding/gfm.db")

### Get all json files in Duke storage

In [122]:
path_to_json = '/Volumes/data_commons-Ubel/gfm/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

### Define functions

In [13]:
#scrape_tb
def make_scrape_tb(data):
    scrape_tb = pd.DataFrame([i['scrape'] for i in data])
    scrape_tb.to_sql('scrape_tb', conn, if_exists='append', index = False)

#feed_tb
def make_feed_tb(data):
    feed = pd.DataFrame(i['feed']['campaign'] for i in data if i['feed'] is not None)
    feed['url'] = ["https://www.gofundme.com/f/" + s for s in feed['url']]
    feed['city'] = [i['city'] for i in feed['location']]
    feed['country'] = [i['country'] for i in feed['location']]
    feed['postal_code'] = [i['postal_code'] for i in feed['location']]
    feed['bene_id'] = [i['id'] for i in feed['beneficiary']]
    feed['bene_user_id'] = [i['user_id'] for i in feed['beneficiary']]
    feed['bene_person_id'] = [i['person_id'] for i in feed['beneficiary']]
    feed['bene_first_name'] = [i['first_name'] for i in feed['beneficiary']]
    feed['bene_last_name'] = [i['last_name'] for i in feed['beneficiary']]
    feed['bene_is_placeholder'] = [i['is_placeholder_bene'] for i in feed['beneficiary']]
    feed['bene_profile_url'] = [i['profile_url'] for i in feed['beneficiary']]
    feed['campaign_photo_url'] = [i['url'] for i in feed['campaign_photo']]
    feed['team_name'] = [i['name'] if i != {} else None for i in feed['team'] ]
    feed['team_pic_url'] = [i['team_pic_url'] if i != {} else None for i in feed['team']]
    feed['team_media_type'] = [i['media_type'] if i != {} else None for i in feed['team']]
    feed['team_pub_attr'] = [i['public_attributions'] if i != {} else None for i in feed['team']]
    feed['team_invite_limit'] = [i['team_invite_limit'] if i != {} else None for i in feed['team']]
    feed['team_status'] = [i['status'] if i != {} else None for i in feed['team']]
    feed['team_created_date'] = [i['created_at'] if i != {} else None for i in feed['team']]
    feed['team_updated_date'] = [i['updated_at'] if i != {} else None for i in feed['team']]
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    donors['url'] = [i['scrape']['url'] for i in data if i['donor'] is not None]
    donors['donor_resp_status'] = [";".join(map(str,i)) for i in donors['donor_resp_status']]
    del donors['donor_list']
    feed = feed.merge(donors, on="url")
    del feed['beneficiary']
    del feed['charity']
    del feed['campaign_photo']
    del feed['location']
    del feed['tags']
    del feed['business']
    del feed['team']
    del feed['partner']
    feed.to_sql('feed_tb', conn, if_exists='append', index = False)
    return feed

#donation
def make_donation_tb(data):
    #add url to each donation log
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for don in data[i]['donor']['donor_list']:
                don['url'] = url
    #make pandas df
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    #extract donation log
    donation_tb = pd.DataFrame(i for i in chain.from_iterable(donors['donor_list']))
    #rename columns
    donation_tb.columns = ["online_id","don_amt","don_offline","don_anon","don_name","don_date","don_profile","don_verified","url"]
    #insert into sql db
    donation_tb.to_sql('donation_tb', conn, if_exists='append', index = False)
    
#team
def make_team_member_tb(data, feed):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for mem in data[i]['feed']['team_members']:
                mem['url'] = url
    feed_for_team = pd.DataFrame([i['feed'] for i in data if i['feed'] is not None])
    if("True" in feed['is_team']):
        team_member_tb = pd.DataFrame(i for i in chain.from_iterable(feed_for_team['team_members']))
        team_member_tb.columns = ["team_mem_amt", "team_mem_fb", "team_mem_first_name","team_mem_gfm_profile","team_mem_id","team_mem_last_name","team_mem_don_attr","team_mem_profile","team_mem_role","team_mem_status","team_mem_person_id","team_mem_locale","url"]
        team_member_tb.to_sql('team_member_tb', conn, if_exists='append', index = False)
        
#comment
def make_comment_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for mem in data[i]['comment']['comment_list']:
                mem['url'] = url
    comments = pd.DataFrame([i['comment'] for i in data if i['comment'] is not None])
    comment_tb = pd.DataFrame(i for i in chain.from_iterable(comments['comment_list']))
    comment_tb['donation_amount'] = [i['amount'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['is_offline'] = [i['is_offline'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['is_anonymous'] = [i['is_anonymous'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['created_at'] = [i['created_at'] if type(i) == dict else None for i in comment_tb['donation']]
    
    #parse out comment variables
    try:
        comment_tb['comment_id_gfm'] = [i['comment_id'] if type(i) == dict else None for i in comment_tb['comment']]
        comment_tb['comment_text'] = [i['comment'] if type(i) == dict else None for i in comment_tb['comment']]
        comment_tb['status'] = [i['status'] if type(i) == dict else None for i in comment_tb['comment']]
        comment_tb['time_stamp'] = [i['timestamp'] if type(i) == dict else None for i in comment_tb['comment']]
        comment_tb['profile_url'] = [i['profile_url'] if type(i) == dict else None for i in comment_tb['comment']]
        comment_tb['deny_delete'] = [i['deny_delete'] if type(i) == dict else None for i in comment_tb['comment']]
    except:
        comment_tb['comment_id_gfm'] = [i['comment_id'] if len(i)>0 else None for i in comment_tb['comment']]
        comment_tb['comment_text'] = [i['comment'] if len(i)>0 else None for i in comment_tb['comment']]
        comment_tb['status'] = [i['status'] if len(i)>0 else None for i in comment_tb['comment']]
        comment_tb['time_stamp'] = [i['timestamp'] if len(i)>0 else None for i in comment_tb['comment']]
        comment_tb['profile_url'] = [i['profile_url'] if len(i)>0 else None for i in comment_tb['comment']]
        comment_tb['deny_delete'] = [i['deny_delete'] if len(i)>0 else None for i in comment_tb['comment']]
    
    comment_tb['photos'] = [i if type(i) == list else [] for i in comment_tb['photos']]
    comment_tb['photos'] = [i[0]['url'] if len(i) > 0 else None for i in comment_tb['photos']]
    del comment_tb['donation']
    del comment_tb['comment']
    del comment_tb['photos']
    del comment_tb['timestamp']
    #if("amount" in comment_tb.columns):
    #    del comment_tb['amount']
    comment_tb.to_sql('comment_tb', conn, if_exists='append', index = False)
    
#update
def make_update_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for mem in data[i]['update']['update_list']:
                mem['url'] = url
    updates = pd.DataFrame([i['update'] for i in data if i['update'] is not None])
    update_tb = pd.DataFrame(i for i in chain.from_iterable(updates['update_list']))
    if(len(update_tb)>0):
        #extract photo url
        update_tb['photo_url'] = [i[0]['url'] if len(i)> 0 else None for i in update_tb['photos']]
        #remove/reformat data in dictionary format so that it can be inserted into sql
        del update_tb['photos']
        #rename columns
        update_tb = update_tb.rename(columns={'text': 'update_text'})
        #send to db
        update_tb.to_sql('update_tb', conn, if_exists='append', index = False)

### Add data to database

In [124]:
for json_file in json_files:
    json_file_path = os.path.join(path_to_json, json_file)
    with open (json_file_path, "r") as f:
        data = json.load(f)
    #send to db
    make_scrape_tb(data)
    feed = make_feed_tb(data)
    make_donation_tb(data)
    make_team_member_tb(data, feed)
    make_comment_tb(data)
    make_update_tb(data)
    print(json_file)

data_301.json
data_197.json
data_11.json
data_474.json
data_406.json
data_444.json
data_164.json
data_322.json
data_319.json
data_100.json
data_37.json
data_85.json
data_4.json
data_10.json
data_278.json
data_154.json
data_103.json
data_321.json
data_382.json
data_91.json
data_22.json
data_135.json
data_109.json
data_27.json
data_451.json
data_369.json
data_137.json
data_172.json
data_361.json
data_386.json
data_15.json
data_478.json
data_170.json
data_279.json
data_123.json
data_20.json
data_250.json
data_455.json
data_359.json
data_449.json
data_304.json
data_17.json
data_18.json
data_237.json
data_52.json
data_223.json
data_146.json
data_446.json
data_330.json
data_176.json
data_74.json
data_47.json
data_125.json
data_214.json
data_87.json
data_495.json
data_89.json
data_262.json
data_347.json
data_477.json
data_241.json
data_54.json
data_393.json
data_227.json
data_469.json
data_236.json
data_281.json
data_112.json
data_481.json
data_209.json
data_65.json
data_129.json
data_277.jso

### ------------------------
### Older code to develop functions above
### ------------------------

In [58]:
#import data
with open('data_474.json') as json_file:
    data = json.load(json_file)

In [29]:
#import data
with open('/Volumes/data_commons-Ubel/gfm/data_462.json') as json_file:
    data = json.load(json_file)

In [12]:
path_to_json = '/Volumes/data_commons-Ubel/gfm/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
json_files = json_files[:2]

In [13]:
json_list = []
for json_file in json_files:
    json_file_path = os.path.join(path_to_json, json_file)
    with open (json_file_path, "r") as f:
        data = json.load(f)
    json_list.append(len(data))
json_files = pd.DataFrame(json_files)
json_list = pd.DataFrame(json_list)
df = pd.concat([json_files, json_list], axis=1)
df.to_csv("files.csv")

In [41]:
json_data = list()
for json_file in json_files:
    json_file_path = os.path.join(path_to_json, json_file)
    with open (json_file_path, "r") as f:
        json_data.append(json.load(f))

KeyboardInterrupt: 

In [87]:
df = pd.read_sql_query("SELECT * FROM feed_tb", conn)

In [88]:
df

,feed_id,id,campaign_id,auto_fb_post_mode,has_beneficiary,category_id,charity_id,is_charity,currencycode,current_amount,...,team_pic_url,team_media_type,team_pub_attr,team_invite_limit,team_status,team_created_date,team_updated_date,donor_resp_status,donor_reached_max,donor_data_error
0,1,8598411,8598411,1,0,11,0,0,USD,2200.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
1,2,14914981,14914981,0,1,11,0,0,USD,1600.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
2,3,41905178,41905178,0,1,11,0,0,USD,2370.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
3,4,11941789,11941789,0,0,11,0,0,USD,8790.0,...,None,NaN,None,NaN,NaN,None,None,200;200,0,0
4,5,2739533,2739533,0,0,11,0,0,USD,1650.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,377,19036438,19036438,1,1,11,0,0,USD,1695.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
377,378,459773,459773,1,0,11,0,0,USD,6205.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
378,379,6964693,6964693,1,0,11,0,0,USD,5125.0,...,None,NaN,None,NaN,NaN,None,None,200,0,0
379,380,1924221,1924221,1,0,11,0,0,USD,10385.0,...,None,NaN,None,NaN,NaN,None,None,200;200,0,0


### Formatting scrape_tb

In [22]:
def make_scrape_tb(data):
    scrape_tb = pd.DataFrame([i['scrape'] for i in data])
    scrape_tb.to_sql('scrape_tb', conn, if_exists='append', index = False)

In [23]:
scrape_tb = pd.DataFrame([i['scrape'] for i in data])

In [21]:
conn = sqlite3.connect("gfm.db")

In [922]:
scrape_tb.to_sql('scrape_tb', conn, if_exists='append', index = False)

In [24]:
scrape_tb

,url,resp_status,date_scrape,cat,target_cat,activity_status,country
0,https://www.gofundme.com/f/Jeremiah29-11,200,"Wed, 19 Aug 2020 17:51:56 GMT",11.0,1.0,campaignlite,US
1,https://www.gofundme.com/f/financement-pour-le...,200,"Wed, 19 Aug 2020 17:51:56 GMT",8.0,0.0,active,CA
2,https://www.gofundme.com/f/help-roulan-get-to-...,200,"Wed, 19 Aug 2020 17:51:57 GMT",17.0,0.0,active,US
3,https://www.gofundme.com/f/gbj300,200,"Wed, 19 Aug 2020 17:51:57 GMT",11.0,1.0,campaignlite,US
4,https://www.gofundme.com/f/khbnjm-knee-injury-...,200,"Wed, 19 Aug 2020 17:51:57 GMT",11.0,1.0,active,CA
...,...,...,...,...,...,...,...
995,https://www.gofundme.com/f/susanstariha,200,"Wed, 19 Aug 2020 17:55:03 GMT",11.0,1.0,active,US
996,https://www.gofundme.com/f/sherririnehart,200,"Wed, 19 Aug 2020 17:54:50 GMT",11.0,1.0,active,US
997,https://www.gofundme.com/f/benjamin-evans-leuk...,200,"Wed, 19 Aug 2020 17:55:04 GMT",11.0,1.0,active,US
998,https://www.gofundme.com/f/medical-help-for-an...,200,"Wed, 19 Aug 2020 17:54:50 GMT",11.0,1.0,active,US


### Formatting feed_tb

In [17]:
def make_feed_tb(data):
    feed = pd.DataFrame(i['feed']['campaign'] for i in data if i['feed'] is not None)
    feed['url'] = ["https://www.gofundme.com/f/" + s for s in feed['url']]
    feed['city'] = [i['city'] for i in feed['location']]
    feed['country'] = [i['country'] for i in feed['location']]
    feed['postal_code'] = [i['postal_code'] for i in feed['location']]
    feed['bene_id'] = [i['id'] for i in feed['beneficiary']]
    feed['bene_user_id'] = [i['user_id'] for i in feed['beneficiary']]
    feed['bene_person_id'] = [i['person_id'] for i in feed['beneficiary']]
    feed['bene_first_name'] = [i['first_name'] for i in feed['beneficiary']]
    feed['bene_last_name'] = [i['last_name'] for i in feed['beneficiary']]
    feed['bene_is_placeholder'] = [i['is_placeholder_bene'] for i in feed['beneficiary']]
    feed['bene_profile_url'] = [i['profile_url'] for i in feed['beneficiary']]
    feed['campaign_photo_url'] = [i['url'] for i in feed['campaign_photo']]
    feed['team_name'] = [i['name'] if i != {} else None for i in feed['team'] ]
    feed['team_pic_url'] = [i['team_pic_url'] if i != {} else None for i in feed['team']]
    feed['team_media_type'] = [i['media_type'] if i != {} else None for i in feed['team']]
    feed['team_pub_attr'] = [i['public_attributions'] if i != {} else None for i in feed['team']]
    feed['team_invite_limit'] = [i['team_invite_limit'] if i != {} else None for i in feed['team']]
    feed['team_status'] = [i['status'] if i != {} else None for i in feed['team']]
    feed['team_created_date'] = [i['created_at'] if i != {} else None for i in feed['team']]
    feed['team_updated_date'] = [i['updated_at'] if i != {} else None for i in feed['team']]
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    donors['url'] = [i['scrape']['url'] for i in data if i['donor'] is not None]
    donors['donor_resp_status'] = [";".join(map(str,i)) for i in donors['donor_resp_status']]
    del donors['donor_list']
    feed = feed.merge(donors, on="url")
    del feed['beneficiary']
    del feed['charity']
    del feed['campaign_photo']
    del feed['location']
    del feed['tags']
    del feed['business']
    del feed['team']
    del feed['partner']
    feed.to_sql('feed_tb', conn, if_exists='append', index = False)

In [25]:
#create table of feed data
feed = pd.DataFrame(i['feed']['campaign'] for i in data if i['feed'] is not None)

In [26]:
#reformat url to full url
feed['url'] = ["https://www.gofundme.com/f/" + s for s in feed['url']]

In [27]:
#parse out location variables into separate columns
feed['city'] = [i['city'] for i in feed['location']]
feed['country'] = [i['country'] for i in feed['location']]
feed['postal_code'] = [i['postal_code'] for i in feed['location']]

In [28]:
#parse out beneficiary variables into separate columns
feed['bene_id'] = [i['id'] for i in feed['beneficiary']]
feed['bene_user_id'] = [i['user_id'] for i in feed['beneficiary']]
feed['bene_person_id'] = [i['person_id'] for i in feed['beneficiary']]
feed['bene_first_name'] = [i['first_name'] for i in feed['beneficiary']]
feed['bene_last_name'] = [i['last_name'] for i in feed['beneficiary']]
feed['bene_is_placeholder'] = [i['is_placeholder_bene'] for i in feed['beneficiary']]
feed['bene_profile_url'] = [i['profile_url'] for i in feed['beneficiary']]

In [29]:
#extract campaign photo url
feed['campaign_photo_url'] = [i['url'] for i in feed['campaign_photo']]

In [30]:
#parse out team variables into separate columns
feed['team_name'] = [i['name'] if i != {} else None for i in feed['team'] ]
feed['team_pic_url'] = [i['team_pic_url'] if i != {} else None for i in feed['team']]
feed['team_media_type'] = [i['media_type'] if i != {} else None for i in feed['team']]
feed['team_pub_attr'] = [i['public_attributions'] if i != {} else None for i in feed['team']]
feed['team_invite_limit'] = [i['team_invite_limit'] if i != {} else None for i in feed['team']]
feed['team_status'] = [i['status'] if i != {} else None for i in feed['team']]
feed['team_created_date'] = [i['created_at'] if i != {} else None for i in feed['team']]
feed['team_updated_date'] = [i['updated_at'] if i != {} else None for i in feed['team']]

In [31]:
#add donation response status to feed_tb
donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
donors['url'] = [i['scrape']['url'] for i in data if i['donor'] is not None]
donors['donor_resp_status'] = [";".join(map(str,i)) for i in donors['donor_resp_status']]
del donors['donor_list']
feed = feed.merge(donors, on="url")

In [32]:
#remove/reformat data in dictionary format so that it can be inserted into sql
del feed['beneficiary']
del feed['charity']
del feed['campaign_photo']
del feed['location']
del feed['tags']
del feed['business']
del feed['team']
del feed['partner']

In [64]:
#insert into sql db
feed.to_sql('feed_tb', conn, if_exists='append', index = False)

NameError: name 'conn' is not defined

In [33]:
feed

,id,campaign_id,auto_fb_post_mode,has_beneficiary,category_id,charity_id,is_charity,currencycode,current_amount,default_url,...,team_pic_url,team_media_type,team_pub_attr,team_invite_limit,team_status,team_created_date,team_updated_date,donor_resp_status,donor_reached_max,donor_data_error
0,49623540,49623540,False,True,11,0,False,USD,7925,ug7zy-noe-rodriguez,...,None,None,True,150.0,1.0,2020-07-20T22:52:17-05:00,2020-07-20T22:52:17-05:00,200,0,0
1,13100439,13100439,False,True,11,0,False,USD,8105,2fj2at94,...,None,None,None,NaN,NaN,None,None,200;200,0,0
2,32481538,32481538,False,True,11,0,False,USD,2775,support-corys-recovery,...,None,None,True,150.0,1.0,2018-08-27T16:55:05-05:00,2018-08-27T16:55:05-05:00,200,0,0
3,12530043,12530043,False,True,11,0,False,USD,11325,2bpv677b,...,None,None,None,NaN,NaN,None,None,200,0,0
4,28325654,28325654,True,False,11,0,False,USD,7334,cordova-stabbing-victim-rj-mrazik,...,None,None,None,NaN,NaN,None,None,200;200,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,5349954,5349954,False,True,11,0,False,USD,2780,zqtwk4c,...,None,None,None,NaN,NaN,None,None,200,0,0
191,7092013,7092013,True,False,11,0,False,USD,25328,d5bduvtw,...,None,None,None,NaN,NaN,None,None,200;200;200,0,0
192,39615096,39615096,False,True,11,0,False,USD,7020,benjamin-evans-leukemia-fight,...,https://d2g8igdw686xgo.cloudfront.net/39615096...,3,True,150.0,1.0,2019-05-31T16:16:03-05:00,2019-05-31T16:19:50-05:00,200;200,0,0
193,48602444,48602444,False,False,11,0,False,USD,33517,medical-help-for-andrea,...,None,None,None,NaN,NaN,None,None,200;200;200;200;200;200;200;200;200;200,1,0


### Formatting donation_tb

In [ ]:
def make_donation_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for don in data[i]['donor']['donor_list']:
                don['url'] = url
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    donation_tb = pd.DataFrame(i for i in chain.from_iterable(donors['donor_list']))
    donation_tb.columns = ["don_id","don_amt","don_offline","don_anon","don_name","don_date","don_profile","don_verified","url"]
    donation_tb.to_sql('donation_tb', conn, if_exists='append', index = False)
    #add url to each donation log
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for don in data[i]['donor']['donor_list']:
                don['url'] = url
    #make pandas df
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    #extract donation log
    donation_tb = pd.DataFrame(i for i in chain.from_iterable(donors['donor_list']))
    #rename columns
    donation_tb.columns = ["don_id","don_amt","don_offline","don_anon","don_name","don_date","don_profile","don_verified","url"]
    #insert into sql db
    donation_tb.to_sql('donation_tb', conn, if_exists='append', index = False)

In [44]:
def make_donation_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for don in data[i]['donor']['donor_list']:
                don['url'] = url
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    donation_tb = pd.DataFrame(i for i in chain.from_iterable(donors['donor_list']))
    donation_tb.columns = ["don_id","don_amt","don_offline","don_anon","don_name","don_date","don_profile","don_verified","url"]
    donation_tb.to_sql('donation_tb', conn, if_exists='append', index = False)

In [34]:
#add url to each donation log
for i in range(0, len(data)):
    camp = data[i]
    scrape = camp['scrape']
    if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
        url = scrape['url']
        for don in data[i]['donor']['donor_list']:
            don['url'] = url

In [35]:
#make pandas df
donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)

In [36]:
#extract donation log
donation_tb = pd.DataFrame(i for i in chain.from_iterable(donors['donor_list']))

In [37]:
#rename columns
donation_tb.columns = ["don_id","don_amt","don_offline","don_anon","don_name","don_date","don_profile","don_verified","url"]

In [936]:
#insert into sql db
donation_tb.to_sql('donation_tb', conn, if_exists='append', index = False)

In [38]:
donation_tb

,don_id,don_amt,don_offline,don_anon,don_name,don_date,don_profile,don_verified,url
0,589035200,250.0,False,False,Bryan Crookham,2020-08-10T20:44:23-05:00,,True,https://www.gofundme.com/f/ug7zy-noe-rodriguez
1,587353842,100.0,False,True,Anonymous,2020-08-07T15:24:37-05:00,,True,https://www.gofundme.com/f/ug7zy-noe-rodriguez
2,586941526,100.0,False,False,Rochelle Morton,2020-08-06T22:47:12-05:00,,True,https://www.gofundme.com/f/ug7zy-noe-rodriguez
3,585614866,100.0,False,False,Ann Scheffel,2020-08-04T20:28:35-05:00,,True,https://www.gofundme.com/f/ug7zy-noe-rodriguez
4,584978190,1000.0,False,True,Anonymous,2020-08-03T18:49:26-05:00,,True,https://www.gofundme.com/f/ug7zy-noe-rodriguez
...,...,...,...,...,...,...,...,...,...
12984,32170895,100.0,False,False,Grace Marinilli,2015-06-06T17:48:10-05:00,,True,https://www.gofundme.com/f/wea29bs
12985,32169132,100.0,False,False,Joan Smith,2015-06-06T17:17:33-05:00,,True,https://www.gofundme.com/f/wea29bs
12986,32167867,50.0,False,False,Sue Ramatowski,2015-06-06T16:56:09-05:00,,True,https://www.gofundme.com/f/wea29bs
12987,32165876,25.0,False,False,David Piver,2015-06-06T16:24:10-05:00,,True,https://www.gofundme.com/f/wea29bs


### Formatting team_member_tb

In [45]:
def make_team_member_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for mem in data[i]['feed']['team_members']:
                mem['url'] = url
    if("True" in feed['is_team']):
        team_member_tb = pd.DataFrame(i for i in chain.from_iterable(feed_for_team['team_members']))
        team_member_tb.columns = ["team_mem_amt", "team_mem_fb", "team_mem_first_name","team_mem_gfm_profile","team_mem_id","team_mem_last_name","team_mem_don_attr","team_mem_profile","team_mem_role","team_mem_status","team_mem_person_id","team_mem_locale","url"]
        team_member_tb.to_sql('team_member_tb', conn, if_exists='append', index = False)

In [36]:
#add url to each team_member log
for i in range(0, len(data)):
    camp = data[i]
    scrape = camp['scrape']
    if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
        url = scrape['url']
        for mem in data[i]['feed']['team_members']:
            mem['url'] = url

In [37]:
#create pandas db
feed_for_team = pd.DataFrame([i['feed'] for i in data if i['feed'] is not None])
#check to see if there are any team members
if("True" in feed['is_team']):
    #extract variables
    team_member_tb = pd.DataFrame(i for i in chain.from_iterable(feed_for_team['team_members']))
    team_member_tb.columns = ["team_mem_amt", "team_mem_fb", "team_mem_first_name","team_mem_gfm_profile","team_mem_id","team_mem_last_name","team_mem_don_attr","team_mem_profile","team_mem_role","team_mem_status","team_mem_person_id","team_mem_locale","url"]
    #team_member_tb.to_sql('team_member_tb', conn, if_exists='append', index = False)

### Formatting comment_tb

In [46]:
def make_comment_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for mem in data[i]['comment']['comment_list']:
                mem['url'] = url
    comments = pd.DataFrame([i['comment'] for i in data if i['comment'] is not None])
    comment_tb = pd.DataFrame(i for i in chain.from_iterable(comments['comment_list']))
    comment_tb['donation_amount'] = [i['amount'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['is_offline'] = [i['is_offline'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['is_anonymous'] = [i['is_anonymous'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['created_at'] = [i['created_at'] if type(i) == dict else None for i in comment_tb['donation']]
    comment_tb['comment_id_gfm'] = [i['comment_id'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['comment_text'] = [i['comment'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['status'] = [i['status'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['time_stamp'] = [i['timestamp'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['profile_url'] = [i['profile_url'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['deny_delete'] = [i['deny_delete'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['photos'] = [i if type(i) == list else [] for i in comment_tb['photos']]
    comment_tb['photos'] = [i[0]['url'] if len(i) > 0 else None for i in comment_tb['photos']]
    del comment_tb['donation']
    del comment_tb['comment']
    del comment_tb['photos']
    del comment_tb['timestamp']
    if("amount" in comment_tb.columns):
        del comment_tb['amount']
    comment_tb.to_sql('comment_tb', conn, if_exists='append', index = False)

In [106]:
#add url to each team_member log
for i in range(0, len(data)):
    camp = data[i]
    scrape = camp['scrape']
    if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
        url = scrape['url']
        for mem in data[i]['comment']['comment_list']:
            mem['url'] = url

In [107]:
comments = pd.DataFrame([i['comment'] for i in data if i['comment'] is not None])

In [108]:
comment_tb = pd.DataFrame(i for i in chain.from_iterable(comments['comment_list']))

In [109]:
#parse out donation variables
comment_tb['donation_amount'] = [i['amount'] if type(i) == dict else None for i in comment_tb['donation']]
comment_tb['is_offline'] = [i['is_offline'] if type(i) == dict else None for i in comment_tb['donation']]
comment_tb['is_anonymous'] = [i['is_anonymous'] if type(i) == dict else None for i in comment_tb['donation']]
comment_tb['created_at'] = [i['created_at'] if type(i) == dict else None for i in comment_tb['donation']]

In [100]:
len({"1": 1})

1

In [110]:
#parse out comment variables
try:
    comment_tb['comment_id_gfm'] = [i['comment_id'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['comment_text'] = [i['comment'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['status'] = [i['status'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['time_stamp'] = [i['timestamp'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['profile_url'] = [i['profile_url'] if type(i) == dict else None for i in comment_tb['comment']]
    comment_tb['deny_delete'] = [i['deny_delete'] if type(i) == dict else None for i in comment_tb['comment']]
except:
    comment_tb['comment_id_gfm'] = [i['comment_id'] if len(i)>0 else None for i in comment_tb['comment']]
    comment_tb['comment_text'] = [i['comment'] if len(i)>0 else None for i in comment_tb['comment']]
    comment_tb['status'] = [i['status'] if len(i)>0 else None for i in comment_tb['comment']]
    comment_tb['time_stamp'] = [i['timestamp'] if len(i)>0 else None for i in comment_tb['comment']]
    comment_tb['profile_url'] = [i['profile_url'] if len(i)>0 else None for i in comment_tb['comment']]
    comment_tb['deny_delete'] = [i['deny_delete'] if len(i)>0 else None for i in comment_tb['comment']]

In [111]:
#parse out photo urls
comment_tb['photos'] = [i if type(i) == list else [] for i in comment_tb['photos']]
comment_tb['photos'] = [i[0]['url'] if len(i) > 0 else None for i in comment_tb['photos']]

In [112]:
#remove/reformat data in dictionary format so that it can be inserted into sql
del comment_tb['donation']
del comment_tb['comment']
del comment_tb['photos']
del comment_tb['timestamp']
if("amount" in comment_tb.columns):
    del comment_tb['amount']

In [946]:
#insert into sql 
comment_tb.to_sql('comment_tb', conn, if_exists='append', index = False)

### Formatting update_tb

In [47]:
def make_update_tb(data):
    for i in range(0, len(data)):
        camp = data[i]
        scrape = camp['scrape']
        if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
            url = scrape['url']
            for mem in data[i]['update']['update_list']:
                mem['url'] = url
    updates = pd.DataFrame([i['update'] for i in data if i['update'] is not None])
    update_tb = pd.DataFrame(i for i in chain.from_iterable(updates['update_list']))
    if(len(update_tb)>0):
        #extract photo url
        update_tb['photo_url'] = [i[0]['url'] if len(i)> 0 else None for i in update_tb['photos']]
        #remove/reformat data in dictionary format so that it can be inserted into sql
        del update_tb['photos']
        #rename columns
        update_tb = update_tb.rename(columns={'text': 'update_text'})
        #send to db
        update_tb.to_sql('update_tb', conn, if_exists='append', index = False)

In [46]:
#add url to each team_member log
for i in range(0, len(data)):
    camp = data[i]
    scrape = camp['scrape']
    if(scrape['target_cat'] == 1 and scrape['country'] == "US" and scrape['activity_status'] == "active"):
        url = scrape['url']
        for mem in data[i]['update']['update_list']:
            mem['url'] = url

In [47]:
#create dataframe
updates = pd.DataFrame([i['update'] for i in data if i['update'] is not None])

In [48]:
#parse out update variables
update_tb = pd.DataFrame(i for i in chain.from_iterable(updates['update_list']))

In [49]:
if(len(update_tb)>0):
    #extract photo url
    update_tb['photo_url'] = [i[0]['url'] if len(i)> 0 else None for i in update_tb['photos']]
    #remove/reformat data in dictionary format so that it can be inserted into sql
    del update_tb['photos']
    #rename columns
    update_tb = update_tb.rename(columns={'text': 'update_text'})
    #send to db
    #update_tb.to_sql('update_tb', conn, if_exists='append', index = False)

In [28]:
campaigns = pd.read_csv("data/sitemaps_combined_0.csv")
urls = campaigns.iloc[:,1].to_list()

In [30]:
len(urls)

200000

In [2]:
check_1 = pd.read_csv("checking_length.csv")
check_2 = pd.read_csv("checking_length_2.csv")
check_3 = pd.read_csv("checking_length_3.csv")

In [3]:
check_1.columns = ["asdf", "file", "length_1"]
check_2.columns = ["asdf", "file", "length_2"]
check_3.columns = ["asdf", "file", "length_3"]

In [8]:
new = check_1.merge(check_2, how='inner', on='file')

In [14]:
new.sort_values('length_1').to_csv("new.csv")

In [10]:
check.columns = ["asdf", "file", "length"]

In [7]:
to_redo = [i.split("_")[1][:-5] for i in check_3[check_3['length_3']<1000]['file']]

In [8]:
for file in to_redo:
    print('sudo python3 scrape.py ' + file)

sudo python3 scrape.py 283
sudo python3 scrape.py 465
sudo python3 scrape.py 439
sudo python3 scrape.py 261
sudo python3 scrape.py 308
sudo python3 scrape.py 211
sudo python3 scrape.py 265


In [18]:
path_to_json = '/Users/sd363/Desktop/crowdfunding/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [26]:
for json_file in json_files:
    json_file_path = os.path.join(path_to_json, json_file)
    with open (json_file_path, "r") as f:
        data = json.load(f)
    print(json_file + '_' + str(len(data)))

data_1.json_1000
data_0.json_1000
data_109.json_1000
data_170.json_1000
data_171.json_999
data_91.json_462
data_321.json_1000
data_2.json_1000
data_19.json_1000
data_301.json_1000


In [42]:
with open('data_171.json') as json_file:
    data = json.load(json_file)

In [38]:
def make_feed_tb(data):
    feed = pd.DataFrame(i['feed']['campaign'] for i in data if i['feed'] is not None)
    feed['url'] = ["https://www.gofundme.com/f/" + s for s in feed['url']]
    feed['city'] = [i['city'] for i in feed['location']]
    feed['country'] = [i['country'] for i in feed['location']]
    feed['postal_code'] = [i['postal_code'] for i in feed['location']]
    feed['bene_id'] = [i['id'] for i in feed['beneficiary']]
    feed['bene_user_id'] = [i['user_id'] for i in feed['beneficiary']]
    feed['bene_person_id'] = [i['person_id'] for i in feed['beneficiary']]
    feed['bene_first_name'] = [i['first_name'] for i in feed['beneficiary']]
    feed['bene_last_name'] = [i['last_name'] for i in feed['beneficiary']]
    feed['bene_is_placeholder'] = [i['is_placeholder_bene'] for i in feed['beneficiary']]
    feed['bene_profile_url'] = [i['profile_url'] for i in feed['beneficiary']]
    feed['campaign_photo_url'] = [i['url'] for i in feed['campaign_photo']]
    feed['team_name'] = [i['name'] if i != {} else None for i in feed['team'] ]
    feed['team_pic_url'] = [i['team_pic_url'] if i != {} else None for i in feed['team']]
    feed['team_media_type'] = [i['media_type'] if i != {} else None for i in feed['team']]
    feed['team_pub_attr'] = [i['public_attributions'] if i != {} else None for i in feed['team']]
    feed['team_invite_limit'] = [i['team_invite_limit'] if i != {} else None for i in feed['team']]
    feed['team_status'] = [i['status'] if i != {} else None for i in feed['team']]
    feed['team_created_date'] = [i['created_at'] if i != {} else None for i in feed['team']]
    feed['team_updated_date'] = [i['updated_at'] if i != {} else None for i in feed['team']]
    donors = pd.DataFrame(i['donor'] for i in data if i['donor'] is not None)
    donors['url'] = [i['scrape']['url'] for i in data if i['donor'] is not None]
    donors['donor_resp_status'] = [";".join(map(str,i)) for i in donors['donor_resp_status']]
    del donors['donor_list']
    feed = feed.merge(donors, on="url")
    del feed['beneficiary']
    del feed['charity']
    del feed['campaign_photo']
    del feed['location']
    del feed['tags']
    del feed['business']
    del feed['team']
    del feed['partner']
    return feed

In [43]:
asdf = make_feed_tb(data)

In [44]:
asdf

,id,campaign_id,auto_fb_post_mode,has_beneficiary,category_id,charity_id,is_charity,currencycode,current_amount,default_url,...,team_pic_url,team_media_type,team_pub_attr,team_invite_limit,team_status,team_created_date,team_updated_date,donor_resp_status,donor_reached_max,donor_data_error
0,49076510,49076510,False,True,11,0,False,USD,20782,keepfightingralphy,...,None,None,True,150.0,1.0,2020-06-24T10:48:06-05:00,2020-06-24T13:32:25-05:00,200;200,0,0
1,5124120,5124120,False,False,11,0,False,USD,2400,y8c4dpc,...,None,None,None,NaN,NaN,None,None,200,0,0
2,45816524,45816524,False,False,11,0,False,USD,270,help-jake-raise-5000-for-multiple-choices,...,None,None,None,NaN,NaN,None,None,200,0,0
3,1100462,1100462,False,False,11,0,False,USD,10410,7cp194,...,None,None,None,NaN,NaN,None,None,200,0,0
4,4023626,4023626,True,False,11,0,False,USD,1550,qvt2h4,...,None,None,None,NaN,NaN,None,None,200,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,45998742,45998742,False,True,11,0,False,USD,695,1x5d4o31g0,...,None,None,None,NaN,NaN,None,None,200,0,0
192,28263740,28263740,False,False,11,0,False,USD,1895,make-big-robs-heart-strong-again,...,None,None,None,NaN,NaN,None,None,200,0,0
193,45411376,45411376,False,True,11,0,False,USD,2430,ayudemos-a-patricio-a-vencer-el-cancer,...,None,None,None,NaN,NaN,None,None,200,0,0
194,1282885,1282885,False,False,11,0,False,USD,20015,8kknok,...,None,None,None,NaN,NaN,None,None,200;200;200;200;200;200,0,0
